In [1]:
from system import *
s1 = system('3')

Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...
Solving effective Lindblau master equation ...

System  3  initialized!


In [11]:
show((s1.eff_hamiltonian_gs[7,7])(gamma_g=0 ))

-0.5*Omega^2*(1/(1.0*DE + 0.5*I*gamma_f) + 1/(1.0*DE - 0.5*I*gamma_f) + 1.0*g^2/((1.0*DE + 0.5*I*gamma_f)^2*(-1.0*g^2/(1.0*DE + 0.5*I*gamma_f) - 3.0*g^2/(1.0*De + 0.5*I*gamma) + 0.5*I*kappa_c)) + 1.0*g^2/((1.0*DE - 0.5*I*gamma_f)^2*(-1.0*g^2/(1.0*DE - 0.5*I*gamma_f) - 3.0*g^2/(1.0*De - 0.5*I*gamma) - 0.5*I*kappa_c)))

In [7]:
show(s1.gs_states)

array(['0g000', '0g001', '0g010', '0g011', '0g100', '0g101', '0g110',
       '0g111'], dtype='<U20')